In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Step 1: Generate Synthetic DHT11-like Data
np.random.seed(42)
data_size = 300
temperature = np.random.uniform(15, 35, size=data_size)  # in Celsius
humidity = np.random.uniform(30, 90, size=data_size)      # in Percent

# Step 2: Create Rule-Based Labels
labels = []
for t, h in zip(temperature, humidity):
    if t < 20 or h < 40:
        labels.append("Cold")
    elif 20 <= t <= 28 and 40 <= h <= 70:
        labels.append("Comfort")
    else:
        labels.append("Hot")

# Create DataFrame
df = pd.DataFrame({
    "temperature": temperature,
    "humidity": humidity,
    "label": labels
})

# Step 3: Encode Labels
label_encoder = LabelEncoder()
df["label_encoded"] = label_encoder.fit_transform(df["label"])

# Step 4: Prepare Features and Target
X = df[["temperature", "humidity"]].values
y = tf.keras.utils.to_categorical(df["label_encoded"].values, num_classes=3)

# Step 5: Scale Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Step 6: Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Step 7: Build TensorFlow Model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Step 8: Train Model
model.fit(X_train, y_train, validation_split=0.2, epochs=50, verbose=1)

# Step 9: Evaluate and Save Model
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy:.2f}")

model.save("model.h5")
print("model.h5 saved successfully!")


c:\Users\white\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 135ms/step - accuracy: 0.4192 - loss: 1.0558 - val_accuracy: 0.5417 - val_loss: 1.0238
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.5900 - loss: 1.0320 - val_accuracy: 0.7083 - val_loss: 1.0005
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.6868 - loss: 1.0088 - val_accuracy: 0.7083 - val_loss: 0.9776
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.6474 - loss: 0.9905 - val_accuracy: 0.7292 - val_loss: 0.9549
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.7118 - loss: 0.9583 - val_accuracy: 0.7917 - val_loss: 0.9323
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.6775 - loss: 0.9520 - val_accuracy: 0.7917 - val_loss: 0.9098
Epoch 7/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6861 - loss: 0.9333 - val_accuracy: 0.7917 - val_loss: 0.8874
Epoch 8/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.7281 - loss: 0.9002 - val_accuracy: 0.7917 - val_loss: 0.8652

Test Accuracy: 0.77
model.h5 saved successfully!


In [2]:
# Step 11: Convert to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Step 12: Save the .tflite file
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

print("model.tflite saved successfully for ESP32.")



INFO:tensorflow:Assets written to: C:\Users\white\AppData\Local\Temp\tmprolaun0h\assets


INFO:tensorflow:Assets written to: C:\Users\white\AppData\Local\Temp\tmprolaun0h\assets


Saved artifact at 'C:\Users\white\AppData\Local\Temp\tmprolaun0h'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 2), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  2135904820048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2135904821200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2135904815824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2135904819088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2135904821776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2135904817744: TensorSpec(shape=(), dtype=tf.resource, name=None)
model.tflite saved successfully for ESP32.


In [4]:
def bin_to_c_array(input_file, output_file, array_name="model_tflite"):
    with open(input_file, "rb") as f:
        data = f.read()

    with open(output_file, "w") as f:
        f.write(f"const unsigned char {array_name}[] = {{\n")
        for i in range(0, len(data), 12):
            chunk = data[i:i+12]
            line = ", ".join(f"0x{b:02x}" for b in chunk)
            f.write("  " + line + ",\n")
        f.write("};\n")
        f.write(f"const unsigned int {array_name}_len = {len(data)};\n")

# Usage example:
bin_to_c_array(r"D:\iot project\model.tflite", "model_data.h")
